In [1]:
%matplotlib inline
#basic python packages for plotting and array management
import numpy as np
import matplotlib.pyplot as plt
import random
import seaborn as sns
#import statsmodels.api as sm
from datetime import datetime

#for data import
import pandas as pd
from __future__ import division
import os
import sys
from importlib import reload
reload(sys)

#Preprocessing
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.decomposition import PCA

#Classifiers
from sklearn.linear_model import LogisticRegression, RidgeClassifier, RidgeClassifierCV, PassiveAggressiveClassifier, Perceptron, SGDClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier, NearestCentroid
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier
from sklearn.neural_network import MLPClassifier

#Ensemble methods and crossvalidation
from sklearn.model_selection import GridSearchCV, cross_val_predict, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.ensemble import AdaBoostClassifier, BaggingClassifier, ExtraTreesClassifier, GradientBoostingClassifier,RandomForestClassifier, VotingClassifier

#PostProcessing
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

import time

In [2]:
customers = pd.read_csv("customers6.csv")
products = pd.read_csv("products.csv")
X_train = pd.read_csv("X_train.csv")
X_test   = pd.read_csv("X_test.csv")
y_train = pd.read_csv("y_train.csv")

# CUSTOMERS

In [3]:
customers.head()

,CustomerId,CountryISOCode,BirthDate,Gender,FirstOrderDate
0,17860216.0,GB,22/10/2015 15:00,Femme,17/10/2015 12:58
1,10588204.0,FR,20/10/2015 19:46,Femme,20/10/2015 00:07
2,17841489.0,DE,20/10/2015 16:40,Homme,12/10/2015 23:04
3,17849688.0,DE,15/10/2015 14:17,Homme,22/10/2015 10:41
4,12916644.0,FR,13/10/2015 15:54,Femme,15/03/2012 10:04


### BirthDate -> Age

In [4]:
customers['BirthDate'] = customers['BirthDate'].apply(lambda x : datetime.strptime(x, '%d/%m/%Y %H:%M') )
customers['Age'] = 2018 - customers['BirthDate'].apply(lambda x : x.year)

In [5]:
for i in range(10,60,5):
    customers['Age' + str(i) + '-' + str(i+5)] = ((customers['Age']>i) & (customers['Age']<=i+5)).astype('float')

In [6]:
customers.head()

,CustomerId,CountryISOCode,BirthDate,Gender,FirstOrderDate,Age,Age10-15,Age15-20,Age20-25,Age25-30,Age30-35,Age35-40,Age40-45,Age45-50,Age50-55,Age55-60
0,17860216.0,GB,2015-10-22 15:00:00,Femme,17/10/2015 12:58,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,10588204.0,FR,2015-10-20 19:46:00,Femme,20/10/2015 00:07,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,17841489.0,DE,2015-10-20 16:40:00,Homme,12/10/2015 23:04,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,17849688.0,DE,2015-10-15 14:17:00,Homme,22/10/2015 10:41,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,12916644.0,FR,2015-10-13 15:54:00,Femme,15/03/2012 10:04,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Gender -> Binaire

In [7]:
## 1 si une femme
customers['Sexe'] = customers['Gender'].eq('Femme').astype('float')

### FirstOrderDate -> Time in Year since first Order

In [8]:
customers['FirstOrderDate'] = customers['FirstOrderDate'].apply(lambda x : datetime.strptime(x, '%d/%m/%Y %H:%M') )
customers['FirstOrder'] = 2018 - customers['FirstOrderDate'].apply(lambda x : x.year)

In [9]:
for i in range(3,10):
    customers['FirstOrder' + '-' + str(i) + 'Y'] = (customers['FirstOrder']==i).astype('float')
customers['FirstOrder-10Y+'] = (customers['FirstOrder']>=10).astype('float')

In [10]:
def creation(Table,Colonne,Liste):
    for element in Liste:     # On parcourt la liste en entier
        names = []
        for nuance in element:    # On parcourt chaque nuance de chaque sous-liste
            name_col = nuance
            name = Table[Colonne].str.contains(nuance).astype('float')
            names.append(name)
        
        col = pd.Series(0,index = range(Table.shape[0]))
        for column in names :
            col = col+column
        
        Table[name_col] = col
    return ('%d colonne(s) ont étaient ajoutées à la DataFrame' %(len(Liste)))

# ORDERS

In [11]:
orders = X_train

### LineItem -> LineItem binary position

In [12]:
for i in range(1,5):
    orders['LineItem' + '_' + str(i) ] = (orders['LineItem']==i).astype('float')
orders['LineItem_5+'] = (orders['LineItem']>=5).astype('float')

for i in range(1,5):
    X_test['LineItem' + '_' + str(i) ] = (X_test['LineItem']==i).astype('float')
X_test['LineItem_5+'] = (X_test['LineItem']>=5).astype('float')

### OrderStatusLabel -> Binary (Expedie = 1)

In [13]:
orders['OrderStatus'] = orders['OrderStatusLabel'].eq('Expédié').astype('float')

X_test['OrderStatus'] = X_test['OrderStatusLabel'].eq('Expédié').astype('float')

### OrderTypelabel -> Binary (DIRECT = 1)

In [14]:
orders['OrderType'] = orders['OrderTypelabel'].eq('DIRECT').astype('float')

X_test['OrderType'] = X_test['OrderTypelabel'].eq('DIRECT').astype('float')

### SeasonLabel -> Binary (Automne/Hiver = 1)

In [15]:
orders['Season'] = orders['SeasonLabel'].eq('Automne/Hiver').astype('float')

X_test['Season'] = X_test['SeasonLabel'].eq('Automne/Hiver').astype('float')

### PayementModeLabel -> Binary Payement Mode

In [16]:
Label = [['Carte','maestro'],['PayPal'],['Gratuit'],['iDeal'],['que'],['Bank'],['DHL']]
creation(orders,'PayementModeLabel',Label)
creation(X_test,'PayementModeLabel',Label)

'7 colonne(s) ont étaient ajoutées à la DataFrame'

### CustomerTypeLabel -> Binary Type (Fidelise = 1)

In [17]:
orders['CustomerType'] = orders['CustomerTypeLabel'].eq('Fidélisé').astype('float')
X_test['CustomerType'] = X_test['CustomerTypeLabel'].eq('Fidélisé').astype('float')

### IsoCode : Inchangé

### DeviceTypeLabel  (attention, ND majoritaire)

In [18]:
Label2 = [['Desktop'],['Tablet'],['Mobile']]
creation(orders,'DeviceTypeLabel',Label2)
creation(X_test,'DeviceTypeLabel',Label2)

'3 colonne(s) ont étaient ajoutées à la DataFrame'

### PricingTypeLabel -> Binary Type

In [19]:
Label3 = [['Plein'],['Soldes'],['Avec CP'],['Sans CP'],['Vente Priv']]
creation(orders,'PricingTypeLabel',Label3)
creation(X_test,'PricingTypeLabel',Label3)

'5 colonne(s) ont étaient ajoutées à la DataFrame'

### TotalLineItems -> Same as LineItem

In [20]:
for i in range(1,6):
    orders['TotalLineItems' + '_' + str(i) ] = (orders['TotalLineItems']==i).astype('float')
orders['TotalLineItems_6+'] = (orders['TotalLineItems']>=6).astype('float')

for i in range(1,6):
    X_test['TotalLineItems' + '_' + str(i) ] = (X_test['TotalLineItems']==i).astype('float')
X_test['TotalLineItems_6+'] = (X_test['TotalLineItems']>=6).astype('float')

### OrderCreationDate -> Categories based on order creation hour

In [21]:
orders['OrderCreationDate'] = orders['OrderCreationDate'].apply(lambda x : datetime.strptime(x, '%Y-%m-%d %H:%M:%S') )

X_test['OrderCreationDate'] = X_test['OrderCreationDate'].apply(lambda x : datetime.strptime(x, '%Y-%m-%d %H:%M:%S') )

In [22]:
temp = orders['OrderCreationDate'].apply(lambda x : x.hour)

orders['Hour_21H_1H'] = ((temp<=1) | (temp>=21)).astype('float')

temp = X_test['OrderCreationDate'].apply(lambda x : x.hour)

X_test['Hour_21H_1H'] = ((temp<=1) | (temp>=21)).astype('float')

In [23]:
# delete columns 
del orders['OrderNumber']
#del orders['VariantId'] # Necssaire pour merge orders/products
del orders['LineItem']
#del orders['CustomerId'] # Necessaire pour merge orders/customers
del orders['OrderStatusLabel']
del orders['OrderTypelabel']
del orders['SeasonLabel']
del orders['PayementModeLabel']
del orders['CustomerTypeLabel']
del orders['DeviceTypeLabel']
del orders['PricingTypeLabel']
del orders['TotalLineItems']
del orders['OrderCreationDate']
del orders['OrderShipDate']
del orders['BillingPostalCode']
# del orders['OrderNumCustomer']  # Le garder et voir statistiques dessus, pas traité.

In [24]:
# delete columns 
del X_test['OrderNumber']
#del X_test['VariantId'] # Necssaire pour merge X_test/products
del X_test['LineItem']
#del X_test['CustomerId'] # Necessaire pour merge X_test/customers
del X_test['OrderStatusLabel']
del X_test['OrderTypelabel']
del X_test['SeasonLabel']
del X_test['PayementModeLabel']
del X_test['CustomerTypeLabel']
del X_test['DeviceTypeLabel']
del X_test['PricingTypeLabel']
del X_test['TotalLineItems']
del X_test['OrderCreationDate']
del X_test['OrderShipDate']
del X_test['BillingPostalCode']
# del X_test['OrderNumCustomer']  # Le garder et voir statistiques dessus, pas traité.

### OrderShipDate -> on pense que la date d'envoi ne joue pas dans le retour du produit

# Products

In [25]:
products.head()

,VariantId,GenderLabel,MarketTargetLabel,SeasonLabel,SeasonalityLabel,BrandId,UniverseLabel,TypeBrand,ProductId,ProductType,...,UpperHeight,HeelHeight,PurchasePriceHT,IsNewCollection,SubtypeLabel,UpperMaterialLabel,LiningMaterialLabel,OutSoleMaterialLabel,RemovableSole,SizeAdviceDescription
0,728257.0,Homme,Classique,Automne/Hiver,Saisonnier,66.0,DÃ©tente,Standard,17267.0,Baskets,...,NaN,0.0,30.50,0.0,Montantes,NaN,NaN,NaN,False,Prenez une taille en dessous de votre pointure...
1,806356.0,Femme,ND,Automne/Hiver,Saisonnier,842.0,ND,Standard,30824.0,Baskets,...,0.0,0.0,43.00,0.0,Montantes,NaN,NaN,NaN,True,Prenez votre pointure habituelle
2,768790.0,Femme,ND,Automne/Hiver,Reconduit,988.0,Ville,Standard,62475.0,Bottines et boots,...,6.0,3.0,54.90,0.0,Bout pointu,NaN,NaN,NaN,False,Prenez votre pointure habituelle
3,515679.0,Femme,ND,Automne/Hiver,Saisonnier,769.0,Ville,Standard,43983.0,Escarpins,...,0.0,13.0,34.50,0.0,Bout rond,NaN,NaN,NaN,False,Prenez votre pointure habituelle
4,1025246.0,Femme,ND,Automne/Hiver,Saisonnier,1244.0,ND,Standard,81493.0,Bottines et boots,...,8.0,4.0,43.76,0.0,Bout rond,NaN,NaN,NaN,False,Prenez votre pointure habituelle


#### 1) Fonction qui crée les colonnes nécessaires

In [26]:
def creation(Table,Colonne,Liste):
    for element in Liste:     # On parcourt la liste en entier
        names = []
        for nuance in element:    # On parcourt chaque nuance de chaque sous-liste
            name_col = nuance
            name = Table[Colonne].str.contains(nuance).astype('float')
            names.append(name)
        
        col = pd.Series(0,index = range(Table.shape[0]))
        for column in names :
            col = col+column
        
        Table[name_col] = col
    return ('%d colonne(s) ont étaient ajoutées à la DataFrame' %(len(Liste)))

#### 2) Analyse de la table

In [27]:
products.dtypes

VariantId                float64
GenderLabel               object
MarketTargetLabel         object
SeasonLabel               object
SeasonalityLabel          object
BrandId                  float64
UniverseLabel             object
TypeBrand                 object
ProductId                float64
ProductType               object
SupplierColor             object
ProductColorId           float64
MinSize                  float64
MaxSize                  float64
CalfTurn                 float64
UpperHeight              float64
HeelHeight               float64
PurchasePriceHT          float64
IsNewCollection          float64
SubtypeLabel              object
UpperMaterialLabel        object
LiningMaterialLabel       object
OutSoleMaterialLabel      object
RemovableSole             object
SizeAdviceDescription     object
dtype: object

#### 3) Modification+ajout de colonnes

##### * Gender Label

In [28]:
Label = [['Femme'],['Enfant'],['Homme'],['Sacs'],['Accessoires']]
creation(products,'GenderLabel',Label)

'5 colonne(s) ont étaient ajoutées à la DataFrame'

##### *MarketTargetLabel

In [29]:
Label2 = [['Contemporaine','contemporain'],['Suiveur'],['Fashionistas'],['Classique'],['Confort'],['Baroudeur','Aventurier'],['Skate']]
creation(products,'MarketTargetLabel',Label2)


'7 colonne(s) ont étaient ajoutées à la DataFrame'

##### *SeasonLabel

In [30]:
Label3 = products['SeasonLabel'].eq('Automne/Hiver').astype('float')
products['SeasonLabel'] = Label3

##### *SeasonalityLabel

In [31]:
Label4 = [['Saisonnier'],['Reconduit'],['Permanent']]
creation(products,'SeasonalityLabel',Label4)

'3 colonne(s) ont étaient ajoutées à la DataFrame'

##### *BrandId

In [32]:
## Problème car représenté par des nombres donc ordonés mais il y en a trop

##### *UniverseLabel

In [33]:
Label5 = [['tente'],['Ville'],['Sport'],['Green'],['Luxe'],['Confort'],['Soir']]
creation(products,'UniverseLabel',Label5)

'7 colonne(s) ont étaient ajoutées à la DataFrame'

##### *TypeBrand

In [34]:
Label6 = [['Standard'],['Luxe'],['Super']]
creation(products,'TypeBrand',Label6)

'3 colonne(s) ont étaient ajoutées à la DataFrame'

##### *ProductType

In [35]:
Label7 = [['Baskets'],['Sandales'],['Bottines'],['Escarpin'],['lacets'],['Ballerines'],['sport'],['Bottes']\
          ,['Mocass'],['Tongs'],['Chaussons'],['Mules'],['scratch'],['Sacs'],['Espadrilles'],['Maroquinerie']]
creation(products,'ProductType',Label7)

'16 colonne(s) ont étaient ajoutées à la DataFrame'

##### *SupplierColor

In [36]:
Label8 = [['Rouge','Red','Bordeaux','rouge','red'],['Jaune','Yellow','jaune','yellow'],\
          ['Vert','vert','green','Green'],['Marine','Bleu','BlUE','Navy','bleu','blue'],\
          ['Noir','Black','Nero','noir','black','nero','Schwarz','NOIR'],\
          ['Blanc','Incolore','Beige','blanc','incolor','beige'],\
          ['Gris','Grey','Anthracite','gris','grey','anthracite'],\
          ['Marron','Brown','Taupe','Tan','Cognac','Chocolate','marron','brown','taupe','tan','cognac'],\
          ['Rose','Pink','rose','pink','purple','Purple']]

Label9 = [['Dark','dark','Fonc','fonc'],['Light','light','clair','Clair']]

creation(products,'SupplierColor',Label8)
creation(products,'SupplierColor',Label9)

'2 colonne(s) ont étaient ajoutées à la DataFrame'

##### *ProductColorId

In [37]:
# Faut le del y'a au maximum 20 commande pour un seul ProductColorId

##### *MinSize

In [38]:
Label10 = [35,36,37,38,39,40,41,42,43,200]
for i in (Label10):
    products['MinSize' + str(i)] = (products['MinSize'].eq(i)).astype('float')

products['MinSize' + '18' + '-' + '34'] = ((products['MinSize']>=18) & (products['MinSize']<=34)).astype('float')
products['MinSize' + '44' + '-' + '48'] = ((products['MinSize']>=44) & (products['MinSize']<=48)).astype('float')

##### *MaxSize

In [39]:
Label11 = [35,36,37,38,39,40,41,42,43,44,45,200]
for i in (Label11):
    products['MaxSize' + str(i)] = (products['MaxSize'].eq(i)).astype('float')

products['MaxSize' + '18' + '-' + '34'] = ((products['MaxSize']>=18) & (products['MaxSize']<=34)).astype('float')
products['MaxSize' + '46' + '-' + '48'] = ((products['MaxSize']>=46) & (products['MaxSize']<=48)).astype('float')

##### *CalfTurn

In [40]:
# Soit il y en a un soit non
Label12 = products['CalfTurn'].eq(0).astype('float')
products['CalfTurn'] = Label12

##### *UpperHeight

In [41]:
Label13 = [0,2,5,6,7,8,9,10,30]
for i in (Label13):
    products['UpperHeight' + str(i)] = (products['UpperHeight'].eq(i)).astype('float')

products['UpperHeight' + '31' + '-' + '38'] = ((products['UpperHeight']>=31) & (products['UpperHeight']<=38)).astype('float')

##### *HeelHeight

In [42]:
Label14 = [0,1,2,3,4,5,6,7,8,9,10]
for i in (Label14):
    products['HeelHeight' + str(i)] = (products['HeelHeight'].eq(i)).astype('float')

products['HeelHeightsup' + '11'] = ((products['HeelHeight']>=11)).astype('float')

##### *PurchasePriceHT

In [43]:
# Pas besoin de changer, c'est une relation d'ordre

##### *IsNewCollection

In [44]:
# Déja en Binaire

##### *SubtypeLabel

In [45]:
Label15 = [['rond'],['Basses'],['Montantes'],['scratch'],['lacet'],['bride'],['zip'],\
           ['pointu'],['compens'],['carr'],['ouvert'],['lastiques'],['Derbies'],['pluie'],\
           ['Bateau'],['Salom'],['Fourr'],['aiguille'],['boots'],['Spartiates'],['Richelieus'],\
           ['Babies']]
creation(products,'SubtypeLabel',Label15)

'22 colonne(s) ont étaient ajoutées à la DataFrame'

##### *UpperMaterialLabel

In [46]:
Label16 = [['Cuir'],['Nubuck'],['Textile'],['verni'],['Synth'],['Caoutchouc']]
creation(products,'UpperMaterialLabel',Label16)

'6 colonne(s) ont étaient ajoutées à la DataFrame'

##### *LiningMaterialLabel

In [47]:
Label17 = [['Textile'],['Cuir'],['Synth'],['doubl']]
creation(products,'LiningMaterialLabel',Label16)

'6 colonne(s) ont étaient ajoutées à la DataFrame'

##### *OutSoleMaterialLabel

In [48]:
Label18 = [['Gomme'],['Caoutchouc'],['Synth'],['Cuir']]
creation(products,'OutSoleMaterialLabel',Label17)

'4 colonne(s) ont étaient ajoutées à la DataFrame'

##### *RemovableSole

In [49]:
Label19 = products['RemovableSole'].eq(True).astype('float')
products['RemovableSole'] = Label19

In [50]:
products['RemovableSole'].value_counts()

0.0    444714
1.0     88806
Name: RemovableSole, dtype: int64

##### *SizeAdviceDescription

In [51]:
Label20 = [['chausse normalement','Prenez votre pointure habituelle'],['dessous'],['dessus'],['larges'],['fins']]
creation(products,'SizeAdviceDescription',Label20)

'5 colonne(s) ont étaient ajoutées à la DataFrame'

#### 4) Delete columns

In [52]:
#del products['VariantId'] # Necessaire pour merge orders/products
del products['GenderLabel']
del products['MarketTargetLabel']
del products['SeasonalityLabel']
del products ['UniverseLabel']
del products ['TypeBrand']
del products ['ProductId'] #environ 600 par productId
del products['ProductType']
del products['SupplierColor']
del products['ProductColorId'] #max 20 par productIdcolor
del products['MinSize']
del products['MaxSize']
del products['CalfTurn']
del products['UpperHeight']
del products['HeelHeight']
del products['SubtypeLabel']
del products['UpperMaterialLabel']
del products['LiningMaterialLabel']
del products['OutSoleMaterialLabel']
del products['SizeAdviceDescription']

# del products['BrandId']  # peut être a supprimer essayer de le traiter

In [53]:
products = products.fillna(0)

# MERGING

### Merge X_train

In [54]:
index = pd.DataFrame({'index':np.arange(X_train.shape[0])})
orders['index']=index # Evite les duplicata
y_train['index']=index

In [55]:
orders_products = pd.merge(orders,products,how='left',on='VariantId')
X_data = pd.merge(orders_products,customers,how='left',on='CustomerId')
data = pd.merge(y_train,X_data,how='left',on="index")
data = data.sort_values(by='index')
data.head()

,OrderNumber,LineItem,ReturnQuantity,ReturnQuantityBin,index,VariantId,CustomerId,IsoCode,Quantity,UnitPMPEUR,...,Sexe,FirstOrder,FirstOrder-3Y,FirstOrder-4Y,FirstOrder-5Y,FirstOrder-6Y,FirstOrder-7Y,FirstOrder-8Y,FirstOrder-9Y,FirstOrder-10Y+
0,73521754,1,0.0,0.0,0,439729,12443972,FR,1,"52,64",...,1.0,7.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,73521754,2,0.0,0.0,1,440174,12443972,FR,1,"52,64",...,1.0,7.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,73525226,1,1.0,1.0,2,494501,12443958,FR,1,"13,17",...,1.0,7.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,73529009,1,1.0,1.0,3,439590,12443946,FR,1,"56,4",...,1.0,7.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,73529009,2,1.0,1.0,4,559476,12443946,FR,1,37,...,1.0,7.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


### Merge X_test

In [56]:
test_products = pd.merge(X_test,products,how='left',on='VariantId')
test_data = pd.merge(test_products,customers,how='left',on='CustomerId')
test_data.head()

,VariantId,CustomerId,IsoCode,Quantity,UnitPMPEUR,OrderNumCustomer,IsOnSale,LineItem_1,LineItem_2,LineItem_3,...,Sexe,FirstOrder,FirstOrder-3Y,FirstOrder-4Y,FirstOrder-5Y,FirstOrder-6Y,FirstOrder-7Y,FirstOrder-8Y,FirstOrder-9Y,FirstOrder-10Y+
0,754279,14369241,GB,1,"52,92",3,0.0,1.0,0.0,0.0,...,0.0,5.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,1043714,16121040,DE,1,"32,4355",2,0.0,1.0,0.0,0.0,...,1.0,4.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,302486,16271310,FR,1,"66,65",1,0.0,1.0,0.0,0.0,...,1.0,4.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1300698,16271314,DE,1,"35,72",1,0.0,1.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1392714,16251703,ES,1,"23,86",1,0.0,1.0,0.0,0.0,...,1.0,4.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


### Delete

In [57]:
# del colonne for merging
del data['VariantId']
del data['CustomerId']
del data['OrderNumber']

# Pour tester une premiere fois on supprime ces colonnes
del data['IsoCode']
del data['CountryISOCode']
del data['BrandId']

In [58]:
# del customers columns
del data['Gender']
del data['FirstOrderDate']
del data['FirstOrder']
del data['BirthDate']
del data['Age']

In [ ]:
# del colonne for merging
del test_data['VariantId']
del test_data['CustomerId']
#del test_data['OrderNumber']

In [60]:
# Pour tester une premiere fois on supprime ces colonnes
del test_data['IsoCode']
del test_data['CountryISOCode']
del test_data['BrandId']

In [61]:
# del customers columns
del test_data['Gender']
del test_data['FirstOrderDate']
del test_data['FirstOrder']
del test_data['BirthDate']
del test_data['Age']

In [ ]:
# del colonne for merging
del X_data['VariantId']
del X_data['CustomerId']
#del X_data['OrderNumber']

In [63]:
# Pour tester une premiere fois on supprime ces colonnes
del X_data['IsoCode']
del X_data['CountryISOCode']
del X_data['BrandId']

# del customers columns
del X_data['Gender']
del X_data['FirstOrderDate']
del X_data['FirstOrder']
del X_data['BirthDate']
del X_data['Age']

# Standardize the data

In [64]:
X_data.columns.values

array(['Quantity', 'UnitPMPEUR', 'OrderNumCustomer', 'IsOnSale',
       'LineItem_1', 'LineItem_2', 'LineItem_3', 'LineItem_4',
       'LineItem_5+', 'OrderStatus', 'OrderType', 'Season', 'maestro',
       'PayPal', 'Gratuit', 'iDeal', 'que', 'Bank', 'DHL', 'CustomerType',
       'Desktop', 'Tablet', 'Mobile', 'Plein', 'Soldes', 'Avec CP',
       'Sans CP', 'Vente Priv', 'TotalLineItems_1', 'TotalLineItems_2',
       'TotalLineItems_3', 'TotalLineItems_4', 'TotalLineItems_5',
       'TotalLineItems_6+', 'Hour_21H_1H', 'index', 'SeasonLabel',
       'PurchasePriceHT', 'IsNewCollection', 'RemovableSole', 'Femme',
       'Enfant', 'Homme', 'Sacs', 'Accessoires', 'contemporain', 'Suiveur',
       'Fashionistas', 'Classique', 'Confort', 'Aventurier', 'Skate',
       'Saisonnier', 'Reconduit', 'Permanent', 'tente', 'Ville', 'Sport',
       'Green', 'Luxe', 'Soir', 'Standard', 'Super', 'Baskets', 'Sandales',
       'Bottines', 'Escarpin', 'lacets', 'Ballerines', 'sport', 'Bottes',
       'Moc

In [72]:
X_data = X_data.sort_values(by='index')

In [73]:
y_train = y_train.sort_values(by='index')

# Defining a feature transformation

In [74]:
def funk_mask(d,masked_features):
    " Defining a simple mask over the input data "
    columns_ext = masked_features
    X1 = d.loc[:,[xx for xx in d.columns if xx not in columns_ext]]
    print("1")
    g = lambda x: x.replace(",",".")
    print("2")
    X1.UnitPMPEUR = map(np.float64,(map(g,X1.UnitPMPEUR)))
    print("3")
    columns2bin = [x for x in X1.columns if X1[x].dtype == np.dtype('O')]
    print("4")
    X2 = pd.get_dummies(X1.loc[:,columns2bin])
    print("5")
    X1 = X1.loc[:,[xx for xx in X1.columns if xx not in columns2bin]]
    print("6")
    res = pd.concat([X1,X2],axis=1)
    print("7")
    res = res.fillna(0)
    return(res)

# Applying the mask

In [66]:
features = ("IsoCode")

In [75]:
x1 = funk_mask(X_data,features)
x2 = funk_mask(test_data,features)
seleckt_columns = np.intersect1d(x1.columns,x2.columns)
x1 = x1.loc[:,seleckt_columns]
x2 = x2.loc[:,seleckt_columns]

1
2
3
4
5
6
7
1
2
3
4
5
6
7


# Supervised learning : Logistic regression model

In [76]:
clf = RandomForestClassifier()
clf.fit(x1.iloc[:100000], y_train.ReturnQuantityBin[:100000])
y_tosubmit = clf.predict_proba(x2.loc[:,x1.columns])
score_forest = np.mean(cross_val_score(clf,x1.iloc[:100000], y_train.ReturnQuantityBin[:100000],cv=10))
print(score_forest)

0.765760027062


In [70]:
print(y_tosubmit[:,1].shape)
np.savetxt('y_pred.txt', y_tosubmit[:,1], fmt='%f')

(800468,)


In [77]:
yres = clf.predict_proba(x1.loc[:100000,x1.columns])
print(roc_auc_score(y_train.ReturnQuantityBin.iloc[:100001],yres[:,1]))

0.92411209971
